# <font color='Purple'>Gravitational Wave Generation Array</font>

A Phase Array of dumbells can make a detectable signal...

#### To do:
1. Calculate the dumbell parameters for given mass and frequency
1. How many dumbells?
1. Far-field radiation pattern from many radiators.
1. Beamed GW won't be a plane wave. So what?
1. How much energy is lost to keep it spinning?
1. How do we levitate while spinning?

##### Related work on GW radiation
1. https://www.mit.edu/~iancross/8901_2019A/readings/Quadrupole-GWradiation-Ferrari.pdf
1. Wikipedia article on the GW Quadrupole formula (https://en.wikipedia.org/wiki/Quadrupole_formula)
1. MIT 8.901 lecture on GW radiation (http://www.mit.edu/~iancross/8901_2019A/lec005.pdf)

## <font color='Orange'>Imports, settings, and constants</font>

In [ ]:

import numpy as np
#import matplotlib as mpl
import matplotlib.pyplot as plt
#import multiprocessing as mproc
#import scipy.signal as sig
import scipy.constants as scc
#import scipy.special as scsp
#import sys, time
from scipy.io import loadmat

# http://www.astropy.org/astropy-tutorials/Quantities.html
# http://docs.astropy.org/en/stable/constants/index.html
from astropy import constants as ascon

# Update the matplotlib configuration parameters:
plt.rcParams.update({'text.usetex': False,
                     'lines.linewidth': 4,
                     'font.family': 'serif',
                     'font.serif': 'Georgia',
                     'font.size': 22,
                     'xtick.direction': 'in',
                     'ytick.direction': 'in',
                     'xtick.labelsize': 'medium',
                     'ytick.labelsize': 'medium',
                     'axes.labelsize': 'medium',
                     'axes.titlesize': 'medium',
                     'axes.grid.axis': 'both',
                     'axes.grid.which': 'both',
                     'axes.grid': True,
                     'grid.color': 'xkcd:beige',
                     'grid.alpha': 0.253,
                     'lines.markersize': 12,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 'small',
                     'legend.framealpha': 0.8,
                     'legend.handletextpad': 0.5,
                     'legend.labelspacing': 0.33,
                     'legend.loc': 'best',
                     'figure.figsize': ((12, 8)),
                     'savefig.dpi': 140,
                     'savefig.bbox': 'tight',
                     'pdf.compression': 9})

def setGrid(ax):
    ax.grid(which='major', alpha=0.6)
    ax.grid(which='major', linestyle='solid', alpha=0.6)
    
cList = [(0, 0.1, 0.9),
         (0.9, 0, 0),
         (0, 0.7, 0),
         (0, 0.8, 0.8),
         (1.0, 0, 0.9),
         (0.8, 0.8, 0),
         (1, 0.5, 0),
         (0.5, 0.5, 0.5),
         (0.4, 0, 0.5),
         (0, 0, 0),
         (0.3, 0, 0),
         (0, 0.3, 0)]

G      = scc.G              # N * m**2 / kg**2; gravitational constant
c      = scc.c

## Terrestrial Dumbell (Current Tech)

In [ ]:
sigma_yield = 9000e6       # Yield strength of annealed silicon [Pa]
m_dumb = 100               # mass of the dumbell end [kg]
L_dumb = 10                # Length of the dumbell [m]
r_dumb = 1                 # radius of the dumbell rod [m]

rho_pb = 11.34e3        # density of lead [kg/m^3]
r_ball = ((m_dumb / rho_pb)/(4/3 * np.pi))**(1/3)


f_rot  = 1e3 / 2
lamduh = c / f_rot

v_dumb = 2*np.pi*(L_dumb/2) * f_rot
a_dumb = v_dumb**2 / (L_dumb / 2)
F      = a_dumb * m_dumb

stress = F / (np.pi * r_dumb**2)

print('Ball radius is ' + '{:0.2f}'.format(r_ball) + ' m')
print(r'Acceleration of ball = ' + '{:0.2g}'.format(a_dumb) + r' m/s^2')
print('Stress = ' + '{:0.2f}'.format(stress/sigma_yield) + 'x Yield Stress')

#### Futuristic Dumbell

In [ ]:
sigma_yield = 5000e9    # ultimate tensile strength of ??? [Pa]
m_f = 1000               # mass of the dumbell end [kg]
L_f = 3000               # Length of the dumbell [m]
r_f = 40            # radius of the dumbell rod [m]

rho_pb = 11.34e3        # density of lead [kg/m^3]
r_b = ((m_dumb / rho_pb)/(4/3 * np.pi))**(1/3)


f_f = 37e3 / 2
lamduh_f = c / f_f

v_f = 2*np.pi*(L_f/2) * f_f
a_f = v_f**2 / (L_f / 2)
F = a_f * m_f

stress = F / (np.pi * r_f**2)

print('Ball radius = ' + '{:0.2f}'.format(r_f) + ' m')
print('Acceleration of ball = ' + '{:0.2g}'.format(a_f) + ' m/s**2')
print('Stress = ' + '{:0.2f}'.format(stress/sigma_yield) + 'x Yield Stress')

## <font color='Navy'>Radiation of a dumbell</font>

The dumbell is levitated from its middle point using a magnet. So we can spin it at any frequency without friction.

The quadrupole formula for the strain from this rotating dumbell is:

$\ddot{I} = \omega^2 \frac{M R^2}{2}$

$\ddot{I} = \frac{1}{2} \sigma_{yield}~A~(L_{dumb} / 2)$

The resulting strain is:

$h = \frac{2 G}{c^4 r} \ddot{I}$



In [ ]:
def h_of_f(omega_rotor, M_ball, d_earth_alien, L_rotor):
    
    I_ddot = 1/2 * M_ball * (L_rotor/2)**2 * (omega_rotor**2)
    h = (2*G)/(c**4 * d_earth_alien) * I_ddot
    
    return h


r = 2 * lamduh   # take the distance to be 2 x wavelength

#h_2020   = (2*G)/(c**4 * r) * (1/2 * m_dumb * (L_dumb/2)**2) * (2*np.pi*f_rot)**2
w_rot = 2 * np.pi * f_rot
h_2020 = h_of_f(w_rot, m_dumb, r, L_dumb)

d_ref = c * 3600*24*365 * 1000  # 1000 light years [m]
d = 1 * d_ref

h_2035 = h_of_f(w_rot, m_dumb, d, L_dumb)
print('Strain from a single   (2018) dumbell  is {h:0.3g}  at a distance of {r:0.1f} km'.format(h=h_2020, r=r/1000))
print('Strain from a single   (2018) dumbell  is {h:0.3g} at a distance of {r:0.1f} kilo lt-years'.format(h=h_2035, r=d/d_ref))


r = 2 * lamduh_f   # take the distance to be 2 x wavelength

h_f = (2*G)/(c**4 * r) * (1/2 * m_f * (L_f/2)**2) * (2*np.pi*f_f)**2
h_2345 = h_of_f(2*np.pi*f_f, m_f, d, L_dumb)

N_rotors = 100e6

print("Strain from a single  (alien) dumbell  is {h:0.3g} at a distance of {r:0.1f} kilo lt-years".format(h=h_2345, r=d/d_ref))
print("Strain from many many (alien) dumbells is " + '{:0.3g}'.format(N_rotors*h_2345) + ' at ' + str(1) + ' k lt-yr')

## <font color='Navy'>Phased Array</font>

Beam pattern for a 2D grid of rotating dumbells

Treat them like point sources?

Make an array and add up all the spherical waves